# Telco Customer Churn Prediction Results

## Introduction and Context

## Libraries and Configurations

In [1]:
import sys
from pathlib import Path

# path to project root
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [2]:
import src.utils as ut
import src.preprocess as pp
import src.config as cf
import src.train as tn
import src.evaluate as et

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

project path: C:\Users\Usuario\OneDrive\Documentos\Proyectos Personales\Data Science\Public Portfolio Data Science\241117 Telco Customer Churn
raw data path: C:\Users\Usuario\OneDrive\Documentos\Proyectos Personales\Data Science\Public Portfolio Data Science\241117 Telco Customer Churn\data\raw_data.csv
model path C:\Users\Usuario\OneDrive\Documentos\Proyectos Personales\Data Science\Public Portfolio Data Science\241117 Telco Customer Churn\models\model.pkl
results path C:\Users\Usuario\OneDrive\Documentos\Proyectos Personales\Data Science\Public Portfolio Data Science\241117 Telco Customer Churn\results


## Data Loading

In [3]:
raw_data = ut.load_data(cf.paths['data_raw'])
raw_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Splitting and Preprocessing

### Target - Feature and Train - Test Split

In [4]:
X = raw_data.drop(columns = 'Churn', axis = 1)
y = raw_data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = cf.test_size, random_state = cf.random_state)

### Pipeline Definition

In [5]:
# create preprocessing pipeline

target_preprocessing_pipeline = pp.TargetPreprocess()

feature_preprocessing_pipeline = Pipeline(steps = [
    ('feature_selection', pp.FeatureSelector()),
    ('data_cleaning', pp.DataCleaning()),
    ('feature_engineering', pp.FeatureEngineering()),
    ('outlier_detection', pp.OutlierDetector()),
    ('missing_values', pp.MissingValuesHandler()),
    ('encoding', pp.CategoricalEncoder()),
    ('scaling', pp.Scaling())
])

### Target Variable Preprocessing

In [ ]:
y_train = target_preprocessing_pipeline.fit_transform(y_train)
y_test = target_preprocessing_pipeline.fit_transform(y_test)


### Feature Preprocessing

In [9]:
X_train = feature_preprocessing_pipeline.fit_transform(X_train)
X_test = feature_preprocessing_pipeline.fit_transform(X_test)

binary encoding applied to : gender
binary encoding applied to : PhoneService
binary encoding applied to : MultipleLines
binary encoding applied to : InternetService
binary encoding applied to : OnlineSecurity
binary encoding applied to : PaperlessBilling
ordinal encoding applied to : Contract
one - hot encoding applied to : PaymentMethod
one - hot encoding applied to : OnlineBackup
one - hot encoding applied to : DeviceProtection
one - hot encoding applied to : TechSupport
one - hot encoding applied to : StreamingTV
one - hot encoding applied to : StreamingMovies
binary encoding applied to : gender
binary encoding applied to : PhoneService
binary encoding applied to : MultipleLines
binary encoding applied to : InternetService
binary encoding applied to : OnlineSecurity
binary encoding applied to : PaperlessBilling
ordinal encoding applied to : Contract
one - hot encoding applied to : PaymentMethod
one - hot encoding applied to : OnlineBackup
one - hot encoding applied to : DeviceProte

In [10]:
X_test

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes
941,0811-GSDTP,0.0,0.0,No,Yes,0.180556,0.0,0.0,1.0,0.0,...,0,1,0,0,1,0,0,1,0,0
1404,1970-KKFWL,0.0,0.0,No,No,0.486111,1.0,1.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
5515,2892-GESUL,0.0,0.0,Yes,Yes,0.250000,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
3684,2842-BCQGE,1.0,0.0,No,No,0.597222,1.0,1.0,1.0,0.0,...,0,1,0,0,1,0,0,1,0,0
7017,4807-IZYOZ,0.0,0.0,No,No,0.708333,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,2609-IAICY,0.0,0.0,No,No,0.013889,1.0,1.0,1.0,0.0,...,0,1,0,0,1,0,0,1,0,0
5585,4667-OHGKG,1.0,0.0,No,No,0.013889,1.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
804,3138-BKYAV,1.0,0.0,No,No,0.083333,1.0,0.0,1.0,0.0,...,0,1,0,0,0,0,1,1,0,0
3190,3745-HRPHI,1.0,0.0,Yes,Yes,0.916667,0.0,0.0,1.0,0.0,...,1,0,0,1,0,0,1,0,0,1


## Modeling

### Random Forest Classifier

#### Training

#### Testing

#### Metrics Report

## Results